In [1]:
# 필요한 라이브러리 import
import pandas as pd 
import folium
import json
import numpy as np 
from folium.plugins import MarkerCluster
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams, style, font_manager, rc
import matplotlib as mpl
from folium import plugins

import folium
import plotly.express as px

In [2]:
import chart_studio
chart_studio.tools.set_credentials_file(username='username', api_key='api_key')
import chart_studio.plotly as py
import plotly.express as px

In [3]:
# 한글폰트 설정
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [4]:
# 마이너스 깨질 때
mpl.rcParams['axes.unicode_minus']=False

In [85]:
df11 = pd.read_csv('./읍·면·동별_세대원수별_세대수_20221012110120 (2).csv',encoding='cp949')
df11

,구군별,읍면동별,1인가구수,전체인구수,test
0,강서구,대저1동,2155,3713,NaN
1,강서구,대저2동,1974,3730,NaN
2,강서구,강동동,1494,2540,NaN
3,강서구,명지1동,6077,21203,NaN
4,강서구,명지2동,1976,10374,NaN
...,...,...,...,...,...
200,해운대구,반여4동,1188,5974,NaN
201,해운대구,반송1동,4414,7873,NaN
202,해운대구,반송2동,5736,11938,NaN
203,해운대구,재송1동,4120,14669,NaN


In [15]:
df = pd.read_excel('./군집결과 통합(최종).xlsx')
df

,K-Means,GMM,K-Medoids,역사명,역위도,역경도,일평균승하차(명),1인 가구수(세대),사업체수,일평균(원)
0,0,0,1,가야역,35.155883,129.042817,5799,6291,49,1926082
1,0,0,1,감전역,35.155528,128.991146,6107,5461,57,2535118
2,0,0,1,강서구청,35.211247,128.981756,4251,4630,109,2029127
3,3,2,2,개금역,35.153284,129.020533,13621,2501,36,4717075
4,0,2,2,거제(법원·검찰청),35.188589,129.073941,9546,3093,114,3099526
...,...,...,...,...,...,...,...,...,...,...
95,0,2,1,충렬사(안락),35.199859,129.097636,5398,1851,35,1117126
96,3,2,4,토성,35.100727,129.019776,13846,2110,7,6002230
97,1,1,3,하단,35.106180,128.966803,29080,4414,36,11010043
98,3,2,4,해운대역,35.163672,129.158908,18794,5075,80,8828608


In [16]:
df_1 = df[df['K-Means']==2]
df_2 = df[df['GMM']==3]
df_3 = df[df['K-Medoids']==5]

In [17]:
df1 = df[df['K-Means']==1]
df2 = df[df['GMM']==1]
df3 = df[df['K-Medoids']==3]

In [57]:
df1

,K-Means,GMM,K-Medoids,역사명,역위도,역경도,일평균승하차(명),1인 가구수(세대),사업체수,일평균(원)
5,1,1,3,경성부경대역,35.137585,129.100548,24395,3694,83,10345206
20,1,1,4,남포,35.097953,129.034869,28802,1762,12,9296111
33,1,1,3,동래,35.205641,129.078506,28565,4246,118,10257972
57,1,1,3,부산,35.115224,129.039700,30061,3391,198,11733629
58,1,1,3,부산대,35.229609,129.089358,24167,3939,70,13975371
62,1,1,3,사상역,35.162361,128.984621,33585,2003,18,14117468
69,1,1,3,센텀시티역,35.168827,129.131745,28867,4259,26,14421978
71,1,1,3,수영역,35.165227,129.114713,24200,3710,202,9960743
74,1,1,3,시청,35.179837,129.076642,23612,4827,114,10293857
78,1,1,3,연산,35.186168,129.081534,30035,3420,342,15742971


In [19]:
#좌표 기본값 지정
lat = 35.181784
long = 129.052852
(lat, long)

(35.181784, 129.052852)

In [178]:
#부산 좌표 json
import json,re
path='./HangJeongDong_ver20220701.geojson'
rfile = open(path,'r',encoding='utf8').read()
jsonfile = json.loads(rfile)
json_busan = {'type': 'FeatureCollection'}
json_pick = []
json_dong = []
for item in jsonfile['features']:
    if item['properties']['sidonm']=='부산광역시':
        dong = item['properties']['adm_nm'].split()[-1]
        item['id'] = dong
        json_dong.append(dong)
        json_pick.append(item)
json_busan['features']=json_pick

In [65]:
#json 수정
df11['읍면동별'] = df11['읍면동별'].str.replace('제', '')
df11['읍면동별'] = df11['읍면동별'].str.replace('거', '거제')
df11['읍면동별'] = df11['읍면동별'].str.replace('가야1동', '가야제1동')

In [71]:
geo_path = './busan_gu.json'

import json
geo_json = json.load(open(geo_path, encoding='utf-8'))

In [72]:
geo_json['features'][0]['properties']['name']

'중구'

In [21]:
# colors = ['red','blue','green','purple','oragne','beige','darkblue','darkgreen','darkpurple','white','pink','lightblue','lightgreen','lightgrey','black','grey']
#bins = list(df1["소계"].quantile([0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1 ]))


m = folium.Map([lat, long],tiles='cartodbpositron', zoom_start=11)

# 서면
for i in df_1.index:
    station1_lat = df_1.loc[i, '역위도']
    station1_long = df_1.loc[i, '역경도']
    color = 'red'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, color=color).add_to(m)

for i in df_2.index:
    station1_lat = df_2.loc[i, '역위도']
    station1_long = df_2.loc[i, '역경도']
    color = 'blue'
    
    folium.CircleMarker([station1_lat, station1_long], radius=15, color=color).add_to(m)

for i in df_3.index:
    station1_lat = df_3.loc[i, '역위도']
    station1_long = df_3.loc[i, '역경도']
    color = 'green'
    
    folium.CircleMarker([station1_lat, station1_long], radius=20, color=color).add_to(m)

# 서면 외 1순위 후보지 
for i in df1.index:
    station1_lat = df1.loc[i, '역위도']
    station1_long = df1.loc[i, '역경도']
    color = '#FF7493'
    
    folium.CircleMarker([station1_lat, station1_long], radius=5, color=color).add_to(m)

for i in df2.index:
    station1_lat = df2.loc[i, '역위도']
    station1_long = df2.loc[i, '역경도']
    tooltip = df2.loc[i, '역사명']
    color = '#46D2D2'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, tooltip = tooltip , color=color).add_to(m)

for i in df3.index:
    station1_lat = df3.loc[i, '역위도']
    station1_long = df3.loc[i, '역경도']
    color = '#EE82EE'
    
    folium.CircleMarker([station1_lat, station1_long], radius=15, color=color).add_to(m)


m


In [29]:
m = folium.Map([lat, long],tiles='cartodbpositron', zoom_start=11)

# 서면
for i in df_1.index:
    station1_lat = df_1.loc[i, '역위도']
    station1_long = df_1.loc[i, '역경도']
    color = '#FF3232'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, color=color).add_to(m)

for i in df1.index:
    station1_lat = df1.loc[i, '역위도']
    station1_long = df1.loc[i, '역경도']
    color = '#FF7493'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, color=color).add_to(m)

m

In [30]:
m = folium.Map([lat, long],tiles='cartodbpositron', zoom_start=11)

for i in df_2.index:
    station1_lat = df_2.loc[i, '역위도']
    station1_long = df_2.loc[i, '역경도']
    color = '#1E90FF'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, color=color).add_to(m)

for i in df2.index:
    station1_lat = df2.loc[i, '역위도']
    station1_long = df2.loc[i, '역경도']
    tooltip = df2.loc[i, '역사명']
    color = '#46D2D2'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, tooltip = tooltip , color=color).add_to(m)

m

In [32]:
m = folium.Map([lat, long],tiles='cartodbpositron', zoom_start=11)

for i in df_3.index:
    station1_lat = df_3.loc[i, '역위도']
    station1_long = df_3.loc[i, '역경도']
    color = '#FF5675'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, color=color).add_to(m)

for i in df3.index:
    station1_lat = df3.loc[i, '역위도']
    station1_long = df3.loc[i, '역경도']
    color = '#EE82EE'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, color=color).add_to(m)


m

In [33]:
dfs=pd.read_csv('C:/Users/admin/Desktop/최종/data/부산 지하철 역정보(수정).csv',encoding='cp949')
dfs.head()

,역코드,역명,영문,전화번호,역주소,역위도,역경도,호선
0,95,다대포해수욕장역,Dadaepo Beach,051-678-6195,부산광역시 사하구 다대로 지하 692,35.048670,128.964100,1호선
1,96,다대포항역,Dadaepo Harbor,051-678-6196,부산광역시 사하구 다대로 지하 548,35.057820,128.971300,1호선
2,97,낫개역,Natgae,051-678-6197,부산광역시 사하구 다대로 지하 422,35.065265,128.979873,1호선
3,98,신장림역,Sinjangnim,051-678-6198,부산광역시 사하구 다대로 지하 310,35.074433,128.977041,1호선
4,99,장림역,Jangnim,051-618-6199,부산광역시 사하구 다대로 지하 230,35.081090,128.977500,1호선


In [34]:
dfs_1 = dfs[dfs['호선'] == '1호선']
dfs_1.head()

,역코드,역명,영문,전화번호,역주소,역위도,역경도,호선
0,95,다대포해수욕장역,Dadaepo Beach,051-678-6195,부산광역시 사하구 다대로 지하 692,35.048670,128.964100,1호선
1,96,다대포항역,Dadaepo Harbor,051-678-6196,부산광역시 사하구 다대로 지하 548,35.057820,128.971300,1호선
2,97,낫개역,Natgae,051-678-6197,부산광역시 사하구 다대로 지하 422,35.065265,128.979873,1호선
3,98,신장림역,Sinjangnim,051-678-6198,부산광역시 사하구 다대로 지하 310,35.074433,128.977041,1호선
4,99,장림역,Jangnim,051-618-6199,부산광역시 사하구 다대로 지하 230,35.081090,128.977500,1호선


In [35]:
dfs_2 = dfs[dfs['호선'] == '2호선']
dfs_2.head()

,역코드,역명,영문,전화번호,역주소,역위도,역경도,호선
40,201,장산,Jangsan,051-678-6201,부산광역시 해운대구 해운대로 지하 820,35.169914,129.176986,2호선
41,202,중동,Jung-dong,051-678-6202,부산광역시 해운대구 해운대로 지하 730,35.166700,129.168604,2호선
42,203,해운대,Haeundae,051-678-6203,부산광역시 해운대구 해운대로 지하 626,35.163672,129.158908,2호선
43,204,동백,Dongbaek,051-678-6204,부산광역시 해운대구 해운대로 지하 522,35.161484,129.147897,2호선
44,205,시립미술관,Busan Museum of Art,051-678-6205,부산광역시 해운대구 해운대로 지하 396,35.168844,129.138933,2호선


In [36]:
dfs_3 = dfs[dfs['호선'] == '3호선']
dfs_3.head()

,역코드,역명,영문,전화번호,역주소,역위도,역경도,호선
83,301,수영,Suyeong,051-678-6301,부산광역시 수영구 수영로 지하 677,35.167753,129.114590,3호선
84,302,망미,Mangmi,051-678-6302,부산광역시 수영구 연수로 지하 337,35.171528,129.108225,3호선
85,303,배산,Baesan,051-678-6303,부산광역시 연제구 연수로 지하 229,35.173504,129.095498,3호선
86,304,물만골,Mulmangol,051-678-6304,부산광역시 연제구 월드컵대로 지하 23,35.176808,129.085748,3호선
87,305,연산(3),Yeonsan,051-678-6305,부산광역시 연제구 중앙대로 지하 1101,35.186173,129.081526,3호선


In [37]:
dfs_4 = dfs[dfs['호선'] == '4호선']
dfs_4.head()

,역코드,역명,영문,전화번호,역주소,역위도,역경도,호선
100,401,미남(4),Minam,051-678-6401,부산광역시 동래구 아시아드대로 지하 232,35.207116,129.069172,4호선
101,402,동래(4),Dongnae,051-678-6402,부산광역시 동래구 충렬대로 지하 147,35.204834,129.077082,4호선
102,403,수안,Suan,051-678-6403,부산광역시 동래구 충렬대로 지하 223,35.201828,129.083806,4호선
103,404,낙민,Nangmin,051-678-6404,부산광역시 동래구 충렬대로 지하 285,35.200254,129.090774,4호선
104,405,충렬사,Chungnyeolsa,051-678-6405,부산광역시 동래구 반송로 지하 205,35.199859,129.097636,4호선


In [38]:
dfs_11=dfs_1[['역위도','역경도']]
dfs_12=dfs_2[['역위도','역경도']]
dfs_13=dfs_3[['역위도','역경도']]
dfs_14=dfs_4[['역위도','역경도']]

In [163]:
# colors = ['red','blue','green','purple','oragne','beige','darkblue','darkgreen','darkpurple','white','pink','lightblue','lightgreen','lightgrey','black','grey'] tiles='cartodbpositron'

currents1 = dfs_11
currents2 = dfs_12
currents3 = dfs_13
currents4 = dfs_14

m = folium.Map([lat, long],tiles='cartodbpositron', zoom_start=11)

s_line1 = folium.PolyLine(
    locations = currents1,
    tooltip = 'polyline',
    weight = 3,
    color = '#FF7878'
).add_to(m)


s_line2 = folium.PolyLine(
    locations = currents2,
    tooltip = 'polyline',
    weight = 3,
    color = '#7AF67A'
).add_to(m)


s_line3 = folium.PolyLine(
    locations = currents3,
    tooltip = 'polyline',
    weight = 3,
    color = '#FFCB9C'
).add_to(m)


s_line4 = folium.PolyLine(
    locations = currents4,
    tooltip = 'polyline',
    weight = 3,
    color = '#78EAFF'
).add_to(m)


# 서면
for i in df_1.index:
    station1_lat = df_1.loc[i, '역위도']
    station1_long = df_1.loc[i, '역경도']
    color = '#FF3232'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10,fill = True, color=color).add_to(m)

for i in df1.index:
    station1_lat = df1.loc[i, '역위도']
    station1_long = df1.loc[i, '역경도']
    color = '#FF7493'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10,fill = True, color=color).add_to(m)

m

m.fit_bounds(m.get_bounds())
m


In [164]:
# colors = ['red','blue','green','purple','oragne','beige','darkblue','darkgreen','darkpurple','white','pink','lightblue','lightgreen','lightgrey','black','grey']

currents1 = dfs_11
currents2 = dfs_12
currents3 = dfs_13
currents4 = dfs_14

m = folium.Map([lat, long], tiles='cartodbpositron',zoom_start=11)

s_line1 = folium.PolyLine(
    locations = currents1,
    tooltip = 'polyline',
    weight = 3,
    color = '#FF7878'
).add_to(m)


s_line2 = folium.PolyLine(
    locations = currents2,
    tooltip = 'polyline',
    weight = 3,
    color = '#7AF67A'
).add_to(m)


s_line3 = folium.PolyLine(
    locations = currents3,
    tooltip = 'polyline',
    weight = 3,
    color = '#FFCB9C'
).add_to(m)


s_line4 = folium.PolyLine(
    locations = currents4,
    tooltip = 'polyline',
    weight = 3,
    color = '#78EAFF'
).add_to(m)


for i in df_2.index:
    station1_lat = df_2.loc[i, '역위도']
    station1_long = df_2.loc[i, '역경도']
    color = '#1E90FF'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10,fill = True, color=color).add_to(m)

for i in df2.index:
    station1_lat = df2.loc[i, '역위도']
    station1_long = df2.loc[i, '역경도']
    tooltip = df2.loc[i, '역사명']
    color = '#46D2D2'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10,fill = True, tooltip = tooltip , color=color).add_to(m)


m.fit_bounds(m.get_bounds())
m


In [165]:
# colors = ['red','blue','green','purple','oragne','beige','darkblue','darkgreen','darkpurple','white','pink','lightblue','lightgreen','lightgrey','black','grey']

currents1 = dfs_11
currents2 = dfs_12
currents3 = dfs_13
currents4 = dfs_14

m = folium.Map([lat, long], tiles='cartodbpositron',zoom_start=11)

s_line1 = folium.PolyLine(
    locations = currents1,
    tooltip = 'polyline',
    weight = 3,
    color = '#FF7878'
).add_to(m)



s_line2 = folium.PolyLine(
    locations = currents2,
    tooltip = 'polyline',
    weight = 3,
    color = '#7AF67A'
).add_to(m)



s_line3 = folium.PolyLine(
    locations = currents3,
    tooltip = 'polyline',
    weight = 3,
    color = '#FFCB9C'
).add_to(m)



s_line4 = folium.PolyLine(
    locations = currents4,
    tooltip = 'polyline',
    weight = 3,
    color = '#78EAFF'
).add_to(m)



for i in df_3.index:
    station1_lat = df_3.loc[i, '역위도']
    station1_long = df_3.loc[i, '역경도']
    color = '#FF5675'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10,fill = True, color=color).add_to(m)

for i in df3.index:
    station1_lat = df3.loc[i, '역위도']
    station1_long = df3.loc[i, '역경도']
    color = '#EE82EE'
    
    folium.CircleMarker([station1_lat, station1_long], radius=10, fill = True, color=color).add_to(m)


m.fit_bounds(m.get_bounds())
m


In [177]:
currents1 = dfs_11
currents2 = dfs_12
currents3 = dfs_13
currents4 = dfs_14

m = folium.Map([lat, long], tiles='cartodbpositron',zoom_start=11)

folium.Choropleth(
    geo_data=geo_json,
    data=df11,
    columns=['구군별', 'test'],
    key_on='feature.properties.name',
    fill_color='PuBu',
    fill_opacity=0.5,
    line_opacity=0.2,
    nan_fill_color="#D7F1FA"
).add_to(m)



s_line1 = folium.PolyLine(
    locations = currents1,
    tooltip = 'polyline',
    weight = 3,
    color = '#FF7878'
).add_to(m)


s_line2 = folium.PolyLine(
    locations = currents2,
    tooltip = 'polyline',
    weight = 3,
    color = '#679467'
).add_to(m)



s_line3 = folium.PolyLine(
    locations = currents3,
    tooltip = 'polyline',
    weight = 3,
    color = '#FFCB9C'
).add_to(m)


s_line4 = folium.PolyLine(
    locations = currents4,
    tooltip = 'polyline',
    weight = 3,
    color = '#78EAFF'
).add_to(m)



for i in df_3.index:
    station1_lat = df_3.loc[i, '역위도']
    station1_long = df_3.loc[i, '역경도']
    color = '#EB3232'
    
    folium.CircleMarker([station1_lat, station1_long], radius=5,fill = True, fill_color = color,fill_opacity=50, color=color).add_to(m)

for i in df3.index:
    station1_lat = df3.loc[i, '역위도']
    station1_long = df3.loc[i, '역경도']
    color = '#EB3232'
    
    folium.CircleMarker([station1_lat, station1_long], radius=5, fill = True,fill_color = color,fill_opacity=50, color=color).add_to(m)


m

In [176]:
currents1 = dfs_11
currents2 = dfs_12
currents3 = dfs_13
currents4 = dfs_14

m = folium.Map([lat, long], tiles='cartodbpositron',zoom_start=11)

folium.Choropleth(
    geo_data=geo_json,
    data=df11,
    columns=['구군별', 'test'],
    key_on='feature.properties.name',
    fill_color='PuBu',
    fill_opacity=0.5,
    line_opacity=0.2,
    nan_fill_color="#E1FFE1"
).add_to(m)



s_line1 = folium.PolyLine(
    locations = currents1,
    tooltip = 'polyline',
    weight = 3,
    color = '#FF7878'
).add_to(m)


s_line2 = folium.PolyLine(
    locations = currents2,
    tooltip = 'polyline',
    weight = 3,
    color = '#679467'
).add_to(m)



s_line3 = folium.PolyLine(
    locations = currents3,
    tooltip = 'polyline',
    weight = 3,
    color = '#FFCB9C'
).add_to(m)


s_line4 = folium.PolyLine(
    locations = currents4,
    tooltip = 'polyline',
    weight = 3,
    color = '#78EAFF'
).add_to(m)



for i in df_2.index:
    station1_lat = df_2.loc[i, '역위도']
    station1_long = df_2.loc[i, '역경도']
    color = '#FFE13C'
    
    folium.CircleMarker([station1_lat, station1_long], radius=5,fill = True,fill_color = color,fill_opacity=50, color=color).add_to(m)

for i in df2.index:
    station1_lat = df2.loc[i, '역위도']
    station1_long = df2.loc[i, '역경도']
    tooltip = df2.loc[i, '역사명']
    color = '#FFE13C'
    
    folium.CircleMarker([station1_lat, station1_long], radius=5,fill = True, tooltip = tooltip ,fill_color = color,fill_opacity=50, color=color).add_to(m)


m.fit_bounds(m.get_bounds())
m

In [175]:
currents1 = dfs_11
currents2 = dfs_12
currents3 = dfs_13
currents4 = dfs_14

m = folium.Map([lat, long], tiles='cartodbpositron',zoom_start=11)

folium.Choropleth(
    geo_data=geo_json,
    data=df11,
    columns=['구군별', 'test'],
    key_on='feature.properties.name',
    fill_color='PuBu',
    fill_opacity=100,
    line_opacity=0.2,
    nan_fill_color="#E1FFE1"
).add_to(m)



s_line1 = folium.PolyLine(
    locations = currents1,
    tooltip = 'polyline',
    weight = 3,
    color = '#FF7878'
).add_to(m)


s_line2 = folium.PolyLine(
    locations = currents2,
    tooltip = 'polyline',
    weight = 3,
    color = '#679467'
).add_to(m)



s_line3 = folium.PolyLine(
    locations = currents3,
    tooltip = 'polyline',
    weight = 3,
    color = '#FFCB9C'
).add_to(m)


s_line4 = folium.PolyLine(
    locations = currents4,
    tooltip = 'polyline',
    weight = 3,
    color = '#78EAFF'
).add_to(m)



for i in df_3.index:
    station1_lat = df_3.loc[i, '역위도']
    station1_long = df_3.loc[i, '역경도']
    color = '#0000FF'
    
    folium.CircleMarker([station1_lat, station1_long], radius=5,fill = True,fill_color = color,fill_opacity=50, color=color).add_to(m)

for i in df3.index:
    station1_lat = df3.loc[i, '역위도']
    station1_long = df3.loc[i, '역경도']
    color = '#0000FF'
    
    folium.CircleMarker([station1_lat, station1_long], radius=5, fill = True,fill_color = color,fill_opacity=50, color=color).add_to(m)


m.fit_bounds(m.get_bounds())
m
